<a href="https://colab.research.google.com/github/ps24/parkingCitationsMLAnalysis/blob/master/DataCleanUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  About Data Clean Up
Data clean up will consist of the following operations:

1: Remove series with 6 mil+ missing data --> these series will be removed from the dataframe:
* Meter Id
* Marked Time
* Plate Expiry Date
* VIN

2: Keep the following data:
* Issue Date
* Issue time
* RP State Plate
* Location
* Violation Description

3: Clean up 'Location' series

* Remove rows with NaN (765 rows)
* Clean up 'Location' fomat (remove nonalpha numeric characters such as '! ? *')

4: Add series: create 'Issue_hour' column from 'Issue time' column to list only hour

5: Add series: create 'Issue_Month', 'Issue_Year', 'Issue_Weekday' column from 'Issue Date' column

6: Remove 'Latitude', 'Longitude' columns

7:  Create new dataframe --> save this dataframe as a csv in gDrive

8: Run Geocode API
* Add series: create 'Zip_Code' column from 'Location'
* Add series: create 'Latitude', 'Longitude' columns from 'Location'

9: Create new dataframe --> Save this dataframe as a csv in gDrive

# Package Imports


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#  Data dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost
from sklearn.model_selection import train_test_split
#import xgboost as xgb

%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False

#  Get Data from GDrive

In [0]:
#  Authenticate Google Account for GDrive access (raw data is saved in GDrive)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#  Raw data file gdrive location:  1YnHtW1HVeAaRM2jx1cwWn_Dl5b_NB57k

all_data_downloaded = drive.CreateFile({'id': '1YnHtW1HVeAaRM2jx1cwWn_Dl5b_NB57k'})
all_data_downloaded.GetContentFile('allData.csv')

df_allData = pd.read_csv('allData.csv', dtype={'Ticket number': str, 'VIN': str})


#  Data Assessment

In [5]:
df_allData.head(5)

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0


In [6]:
df_allData.tail(5)

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
8490845,4345072852,2018-11-23T00:00:00,1030.0,7,NaN,CA,NaN,NaN,MERZ,PA,WT,6283 COMMODORE SLOAT DR,00315,51.0,80.58L,PREFERENTIAL PARKING,68.0,6.450392e+06,1.844310e+06
8490846,4345072863,2018-11-23T00:00:00,1039.0,131,NaN,CA,201909.0,NaN,HOND,PA,BK,1942 CRESCENT HEIGHTS BLVD,00315,51.0,80.58L,PREFERENTIAL PARKING,68.0,6.447931e+06,1.837655e+06
8490847,4345072874,2018-11-23T00:00:00,1041.0,131,NaN,IN,201905.0,NaN,FORD,PA,GY,1942 CRESCENT HEIGHTS BLVD,00315,51.0,80.58L,PREFERENTIAL PARKING,68.0,6.447931e+06,1.837655e+06
8490848,4345072885,2018-11-23T00:00:00,1218.0,NaN,NaN,CA,201901.0,NaN,CHEV,TK,BK,1711 CRESCENT HEIGHTS BLVD,00315,51.0,80.69.2,COMM VEH OVER TIME LIMIT,103.0,6.448424e+06,1.839476e+06
8490849,4345072896,2018-11-23T00:00:00,1411.0,NaN,1000.0,CA,201911.0,NaN,HOND,PA,GN,11101 EXPOSITION BLVD,00315,51.0,80.69C,PARKED OVER TIME LIMIT,58.0,6.430737e+06,1.835574e+06


In [7]:
df_allData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
Meter Id                  object
Marked Time              float64
RP State Plate            object
Plate Expiry Date        float64
VIN                       object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                 float64
Longitude                float64
dtype: object

In [8]:
#  Show NaN values
df_allData.isnull().sum()

Ticket number                  0
Issue Date                   458
Issue time                  2433
Meter Id                 6272543
Marked Time              8205539
RP State Plate               765
Plate Expiry Date         771669
VIN                      8474541
Make                        8487
Body Style                  8510
Color                       3916
Location                     765
Route                      63569
Agency                       467
Violation code                 0
Violation Description        839
Fine amount                 6252
Latitude                       3
Longitude                      3
dtype: int64

In [9]:
#  Issue time:  Look at which rows are missing this information
df_allData.shape

(8490850, 19)

# Step 1: Remove series with 6 mil+ missing data fields.  

The following series will be removed from the dataframe:

* Meter Id
* Marked Time
* Plate Expiry Date
* VIN

In [0]:
df_cleanData = df_allData.drop(['Meter Id', 'Marked Time', 'Plate Expiry Date', 'VIN'], axis='columns')

In [11]:
df_cleanData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                 float64
Longitude                float64
dtype: object

In [12]:
df_cleanData.shape

(8490850, 15)

#  Step 2: Keep the following data and remove null values:

* Issue Date
* Issue time
* RP State Plate
* Location
* Violation Description

In [13]:
#  Assess how much impact it would be to remove NULL rows in 'Issue Date', 'Issue time', 'RP State Plate', Location', 'Violation'
df_cleanData.dropna(subset=['Issue Date', 'Issue time', 'RP State Plate', 'Location', 'Violation Description'], how='any').shape


(8486532, 15)

In [14]:
df_cleanData.shape

(8490850, 15)

In [0]:
# Conclusion:  Removing the NaN entries in 'Issue Date', 'Issue time', 'RP State Plate', 'Location', 'Violation Description' only 
# removes 4,318 entries.  Hence,  Remove these NaN rows 

In [0]:
df_cleanData = df_cleanData.dropna(subset=['Issue Date', 'Issue time', 'RP State Plate', 'Location', 'Violation Description'], how='any')

In [17]:
df_cleanData.shape

(8486532, 15)

In [0]:
#  Check that 'Issue Date', 'Issue time', 'RP State Plate', 'Location', 'Violation Description' null rows are dropped

In [19]:
df_cleanData.isnull().sum()

Ticket number                0
Issue Date                   0
Issue time                   0
RP State Plate               0
Make                      7966
Body Style                7942
Color                     3401
Location                     0
Route                    62658
Agency                       9
Violation code               0
Violation Description        0
Fine amount               5410
Latitude                     2
Longitude                    2
dtype: int64

# 3:  Clean up 'Location' series
*  Clean up 'Location' fomat (remove nonalpha numeric characters such as '! ? *')


In [20]:
#  Figure out how many street names start with or contain nonalpha characters
df_cleanData.Location.sort_values()

3490240            ! $ $ HALSEY ST
7543384            ! % CULVER BLVD
3614653          !! * BERENDO ST N
612679          !000 VAN BUREN AVE
3450914               !001 3RD AVE
3450913               !001 3RD AVE
6887925            !015 GAYLEY AVE
6328525        !027 MANCHESTER AVE
5116215        !027 MANCHESTER AVE
6328526        !027 MANCHESTER AVE
6328524        !027 MANCHESTER AVE
6328523        !027 MANCHESTER AVE
5116214        !027 MANCHESTER AVE
5456256        !027 MANCHESTER AVE
6835673       !0341 LOU-DILLON AVE
4670182        !036 HYPERION AVE N
5514490       !0400 LOU-DILLON AVE
6625519           !042 KINGSLEY DR
1401363           !0500 ASHTON AVE
491343          !0801 MISSOURI AVE
6252516           !0818 JUNIPER ST
2602918             !0850 ACAMA ST
2349472         !09 WESTLAKE AVE N
1713092         !0900 MISSOURI AVE
6319629    !0901 MASSACHUSETTS AVE
1495038          !0924 WEYBURN AVE
1993409          !0937 WEYBURN AVE
1809976         !0965 FRUITLAND DR
1809977         !096

In [0]:
#  Observation:  There are 'Location' rows with nonalphanumeric characters (!, $ $ %)
#  Solution:  Remove these nonalphanumeric characters

In [0]:
df_cleanData.Location = df_cleanData.Location.str.replace('\W', ' ')

In [39]:
#  Confirm nonalphanumeric characters are deleted
df_cleanData.Location.sort_values()

3490240               HALSEY ST
694562           SCHOOLCRAFT ST
7664904              101ST ST W
3614653            BERENDO ST N
2492195           WABASH AVENUE
4732510         WHITSETT AVENUE
6938468                4TH ST E
7543384             CULVER BLVD
2594067                 HILL PL
3534543               MAIN ST S
763339              STANLEY AVE
5581122              WINSTON ST
293834          NORTH VENICE BL
293833          NORTH VENICE BL
293835          NORTH VENICE BL
7251038           LEXINGTON AVE
612679        000 VAN BUREN AVE
3450914             001 3RD AVE
3450913             001 3RD AVE
6887925          015 GAYLEY AVE
6328524      027 MANCHESTER AVE
5116215      027 MANCHESTER AVE
6328525      027 MANCHESTER AVE
6328526      027 MANCHESTER AVE
5116214      027 MANCHESTER AVE
5456256      027 MANCHESTER AVE
6328523      027 MANCHESTER AVE
6835673     0341 LOU DILLON AVE
4670182      036 HYPERION AVE N
5514490     0400 LOU DILLON AVE
                   ...         
5027651 

In [40]:
#  Confirm dataFrame has same the number of values
df_cleanData.shape

(8486532, 15)

In [41]:
df_cleanData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                 float64
Longitude                float64
dtype: object

In [42]:
df_cleanData.Location.describe()

count               8486532
unique              1482550
top       1301 ELECTRIC AVE
freq                   9021
Name: Location, dtype: object

In [43]:
df_cleanData.Location.value_counts()

1301 ELECTRIC AVE                  9021
11600 SAN VICENTE BL               6964
101 LARCHMONT BL N                 5918
2377 MIDVALE AVE                   5763
4301 TUJUNGA AV                    5062
1600 IRVING TABOR CT               5061
5901 98TH ST W                     4927
1235 FIGUEROA PL                   4829
12100 VENTURA BL                   4776
4300 TUJUNGA AV                    4704
2800 E OBSERVATORY                 4689
7000 HAWTHORN AVE                  4687
3101 GLENDALE BL                   4360
11100 WEDDINGTON ST                4360
11601 SAN VICENTE BL               4226
100 LARCHMONT BL N                 4101
11001 MCCORMICK ST                 4077
12300 VENTURA BL                   3748
200 LARCHMONT BL N                 3709
1400 HOPE ST S                     3677
12101 VENTURA BL                   3676
11900 SAN VICENTE BL               3650
11700 SAN VICENTE BL               3633
2110 CORINTH AVE                   3629
6041 CADILLAC AVE                  3476


#  Step 4: Add series: create 'Issue_hour' column 

Create 'Issue_hour' column from 'Issue time' column to list only hour (in Military time format)

In [47]:
df_cleanData['Issue time'].head()

0    1251.0
1    1435.0
2    2055.0
3    1515.0
4     115.0
Name: Issue time, dtype: float64

In [48]:
#  Break out 'Issue Time' series to military time (keep hours only)
df_cleanData['Issue_Hour'] = df_cleanData['Issue time'].floordiv(100)
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Issue_Hour
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,12.0
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,14.0
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,20.0
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2,15.0
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,1.0


In [49]:
#  Assess "Issue_Hour" is as expected
df_cleanData['Issue_Hour'].value_counts()

12.0    1082114
8.0     1058151
10.0    1026404
11.0     730913
13.0     530245
9.0      529342
16.0     447163
14.0     405274
18.0     323847
17.0     316788
15.0     295285
19.0     287068
7.0      188698
1.0      169093
20.0     160983
2.0      151387
22.0     146802
21.0     130306
6.0      108424
23.0      91986
3.0       87698
4.0       84245
0.0       78465
5.0       55851
Name: Issue_Hour, dtype: int64

# Step 5: Add series: create 'Issue_Month', 'Issue_Year', 'Issue_Weekday' columns

Create 'Issue_Month', 'Issue_Year', 'Issue_Weekday' columns from 'Issue Date' column

In [50]:
df_cleanData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                 float64
Longitude                float64
Issue_Hour               float64
dtype: object

In [51]:
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Issue_Hour
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,12.0
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,14.0
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,20.0
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2,15.0
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,1.0


In [0]:
#  Add series within the dataframe to break out 'Issue Date' into additional seroes:
#  'Issue_Year', 'Issue_Month', 'Issue_Month_Year'
df_cleanData['Issue_Year'] = pd.DatetimeIndex(df_cleanData['Issue Date']).year
df_cleanData['Issue_Month'] = pd.DatetimeIndex(df_cleanData['Issue Date']).month
df_cleanData['Issue_Month_Year'] = pd.to_datetime(df_cleanData['Issue Date']).dt.to_period('M')
df_cleanData['Issue_Weekday'] = pd.to_datetime(df_cleanData['Issue Date']).dt.weekday_name


In [54]:
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,12.0,2015,12,2015-12,Monday
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,14.0,2015,12,2015-12,Monday
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,20.0,2015,12,2015-12,Monday
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2,15.0,2015,12,2015-12,Saturday
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,1.0,2015,9,2015-09,Tuesday


# Step 6: Remove 'Latitude', 'Longitude' columns

'Latitude' and 'Longitude' columns contain mostly 99999999 values (useless).  These columns will be added appropriately after geocoding

In [55]:
df_cleanData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                 float64
Longitude                float64
Issue_Hour               float64
Issue_Year                 int64
Issue_Month                int64
Issue_Month_Year          object
Issue_Weekday             object
dtype: object

In [56]:
df_cleanData.Latitude.value_counts()

9.999900e+04    1275470
6.463690e+06       8371
6.419829e+06       7084
6.462121e+06       6851
6.458312e+06       6421
6.463690e+06       5643
6.461454e+06       5640
6.448999e+06       5081
6.421106e+06       4870
6.458312e+06       4715
6.463690e+06       4428
6.463690e+06       3739
6.449301e+06       3737
6.457731e+06       3668
6.457731e+06       3528
6.470141e+06       3491
6.449000e+06       3367
6.486945e+06       3339
6.426798e+06       3319
6.421786e+06       3110
6.462121e+06       3089
6.439920e+06       3084
6.447001e+06       3060
6.447001e+06       3056
6.448999e+06       2997
6.419538e+06       2980
6.483074e+06       2957
6.427372e+06       2929
6.427693e+06       2918
6.461452e+06       2913
                 ...   
6.475933e+06          1
6.374763e+06          1
6.454964e+06          1
6.450719e+06          1
6.454979e+06          1
6.488474e+06          1
6.457806e+06          1
6.454994e+06          1
6.474470e+06          1
6.455026e+06          1
6.484937e+06    

In [57]:
df_cleanData.Longitude.value_counts()

9.999900e+04    1275470
1.819688e+06       7084
1.859071e+06       6421
1.882602e+06       5081
1.849114e+06       5038
1.859071e+06       4792
1.849114e+06       4215
1.836817e+06       3667
1.864752e+06       3498
1.819197e+06       3483
1.883364e+06       3367
1.845451e+06       3327
1.837270e+06       3280
1.858229e+06       3220
1.857542e+06       3180
1.840434e+06       3110
1.857542e+06       3089
1.845113e+06       3058
1.845451e+06       3058
1.876409e+06       3056
1.848423e+06       3047
1.882602e+06       2994
1.852081e+06       2859
1.882982e+06       2846
1.743317e+06       2762
1.859358e+06       2750
1.848423e+06       2720
1.845113e+06       2709
1.849337e+06       2699
1.859358e+06       2694
                 ...   
1.849769e+06          1
1.849789e+06          1
1.833999e+06          1
1.849394e+06          1
1.836669e+06          1
1.896851e+06          1
1.849737e+06          1
1.896776e+06          1
1.849746e+06          1
1.894605e+06          1
1.844966e+06    

In [0]:
df_cleanData = df_cleanData.drop(['Longitude', 'Latitude'], axis='columns')

In [61]:
df_cleanData.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Issue_Hour               float64
Issue_Year                 int64
Issue_Month                int64
Issue_Month_Year          object
Issue_Weekday             object
dtype: object

In [62]:
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,12.0,2015,12,2015-12,Monday
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,20.0,2015,12,2015-12,Monday
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,15.0,2015,12,2015-12,Saturday
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,1.0,2015,9,2015-09,Tuesday


#  Step 7: Create new dataframe --> save this dataframe as a csv in gDrive

In [0]:
df_cleanData.to_csv("example.csv")
uploaded = drive.CreateFile({'title': 'cleanData.csv'})
uploaded.SetContentFile("example.csv")
uploaded.Upload()